In [14]:
# Import Statements

import csv
import os
import platform
import sys

# Spark imports
from pyspark.ml.feature import VectorAssembler, VectorSlicer, RobustScaler, UnivariateFeatureSelector
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, OneVsRest
from pyspark.rdd import RDD
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, LongType, StringType
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, Normalizer, VectorSlicer, OneHotEncoder
import findspark
findspark.init()

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

In [2]:
#---Phase 1: Data Loading & Formatting

In [3]:

# Initialize a spark session.
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

def load_df_from_csv(filename):
    spark = init_spark()
    df = spark.read.csv(filename, header=True, multiLine=True, quote="\"", escape="\"")
    return df

# UDFs used for data preprocesssing
def json_list_num(all_cast):
    cleaned = all_cast.replace("/", "")
    converted_list = list(eval(cleaned))
    return len(converted_list)


def gender_of_first_cast(all_cast):
    cleaned = all_cast.replace("/", "")
    converted_list = list(eval(cleaned))
    if converted_list and converted_list[0] and converted_list[0]["gender"]:
        return converted_list[0]["gender"]
    else:
        return None


def gender_of_second_cast(all_cast):
    cleaned = all_cast.replace("/", "")
    converted_list = list(eval(cleaned))
    if converted_list and len(converted_list)>1 and converted_list[1] and converted_list[1]["gender"]:
        return converted_list[1]["gender"]
    else:
        return None


def production_companies(all_production):
    cleaned = all_production.replace("/", "")
    list_of_companies = list(eval(cleaned))
    s = ""
    for company in list_of_companies:
        if len(s) > 0:
            s += "|"
        s += company["name"]
    return s


def release_year(date):
    if date:
        return int(date[:4])
    return date


def release_month(date):
    if date:
        return int(date[5:7])
    return date


def imdb_title(title):
    return title.split("\xa0")[0]


In [4]:

# IMDB database
imdb_dataset = load_df_from_csv("datasets" + ("\\" if platform.system() == "Windows" else "/") + "imdb_movie_metadata.csv")

# remove useless columns from the dataset
imdb_dataset = imdb_dataset.drop("color", "director_name", "director_facebook_likes", "num_critic_for_reviews",
                                 "actor_3_facebook_likes", "actor_2_name", "actor_1_name", "num_voted_users",
                                 "actor_3_name", "facenumber_in_poster", "plot_keywords", "movie_imdb_link",
                                 "num_user_for_reviews", "language", "country", "title_year", "actor_2_facebook_likes",
                                 "aspect_ratio", "actor_1_facebook_likes", "gross")

imdb_dataset = imdb_dataset.withColumnRenamed("movie_title", "imdb_movie_title")

udf_imdb_title = udf(imdb_title, StringType())
imdb_dataset = imdb_dataset.withColumn("imdb_movie_title", udf_imdb_title("imdb_movie_title"))

# TMDB credits database
crew_dataset = load_df_from_csv("datasets/" + "tmdb_5000_credits.csv")

udf_cast_num = udf(json_list_num, IntegerType())
udf_first_cast_gender = udf(gender_of_first_cast, IntegerType())
udf_cast_num = udf(json_list_num, IntegerType())
udf_first_cast_gender = udf(gender_of_first_cast, IntegerType())
udf_second_cast_gender = udf(gender_of_second_cast, IntegerType())

crew_dataset = crew_dataset.withColumn("cast_number", udf_cast_num("cast")) \
    .withColumn("cast_number", udf_cast_num("crew")) \
    .withColumn("first_cast_gender", udf_first_cast_gender("cast")) \
    .withColumn("second_cast_gender", udf_second_cast_gender("cast"))

crew_dataset = crew_dataset.drop("cast", "crew")
crew_dataset = crew_dataset.withColumnRenamed("title", "tmdb_movie_title")

# TMDB Movie dataset
tmdb_dataset = load_df_from_csv("datasets/" + "tmdb_5000_movies.csv")

tmdb_dataset = tmdb_dataset.select("production_companies", "title", "release_date", "vote_average", "revenue", "id")

udf_production_companies = udf(production_companies, StringType())
tmdb_dataset = tmdb_dataset.withColumn("production_companies", udf_production_companies("production_companies"))

udf_release_year = udf(release_year, IntegerType())
udf_release_month = udf(release_month, IntegerType())
tmdb_dataset = tmdb_dataset.withColumn("release_year", udf_release_year("release_date"))
tmdb_dataset = tmdb_dataset.withColumn("release_month", udf_release_month("release_date"))

tmdb_dataset = tmdb_dataset.drop("release_date")

# merge the 3 datasets

two_tmdb_joined = tmdb_dataset.join(crew_dataset, crew_dataset.movie_id == tmdb_dataset.id, "inner").drop("id").drop(
    "movie_id")
dataset = two_tmdb_joined.join(imdb_dataset, two_tmdb_joined.title == imdb_dataset.imdb_movie_title, "inner").drop(
    "imdb_movie_title").drop("tmdb_movie_title")

# remove all null values
cols = dataset.columns
for col in cols:
    dataset = dataset.filter(dataset[str(col)].isNotNull())


In [5]:
# Cast all columns from string to integer type
dataset = (dataset.withColumn("vote_average",(dataset["vote_average"].cast(IntegerType()))))
dataset = (dataset.withColumn("duration",(dataset["duration"].cast(IntegerType()))))
dataset = (dataset.withColumn("cast_total_facebook_likes",(dataset["cast_total_facebook_likes"].cast(IntegerType()))))
dataset = (dataset.withColumn("imdb_score",(dataset["imdb_score"].cast(IntegerType()))))
dataset = (dataset.withColumn("movie_facebook_likes",(dataset["movie_facebook_likes"].cast(IntegerType()))))
dataset = (dataset.withColumn("budget",(dataset["budget"].cast(LongType()))))


In [6]:
#---Phase 2: Data Transformation

In [7]:
# Categorical variable transformations


#  Convert "content rating" variable using One Hot Encoder

# Step 1 String Indexer part
indexer = StringIndexer(inputCol='content_rating', outputCol='ContentIndex')
indexed = indexer.fit(dataset).transform(dataset)

# Step 2:   OneHotEncoding part
encoder = OneHotEncoder(inputCol='ContentIndex', outputCol='OHEContentIndex')
dataset = encoder.fit(indexed).transform(indexed)

dataset = dataset.drop("ContentIndex","content_rating")

In [212]:
# PROCESSING THE GENRES INTO 2 FORMATS:
# 1) StringIndexer + OneHotEncoding => output in file "genres_output/output-noah-index-encoded.csv"
# 2) StringIndexer + OneHotEncoding + VectorAssembler => output in file "genres_output/output-noah-vector.csv"
initial_column_names = ['title', 'genres', 'production_companies', 'vote_average', 'revenue', 'release_year', 'release_month',
                        'cast_number', 'first_cast_gender', 'second_cast_gender', 'duration', 'cast_total_facebook_likes', 'content_rating', 'budget',
                        'imdb_score', 'movie_facebook_likes']
def modify_dataset(dataset):
    dataset = dataset.select('title', 'genres', 'production_companies', 'vote_average', 'revenue', 'release_year', 'release_month',
                            'cast_number', 'first_cast_gender', 'second_cast_gender', 'duration', 'cast_total_facebook_likes', 'content_rating', 'budget',
                            'imdb_score', 'movie_facebook_likes')

    # dataset.show()
    # dataset.toPandas().to_csv('output-dataset.csv')
    return dataset


def get_dataset_movie_genres(dataset):
    ''' Input: dataset (df)
    Output: "list" of all unique genres from the dataset (df)
    '''
    getAllGenres = dataset.select('title', 'genres')
    getAllGenres_rdd = getAllGenres.rdd
    getAllGenres_rdd = getAllGenres_rdd.map(lambda x: (x[0], x[1].split("|")))

    fixedListOfGenres = getAllGenres_rdd.flatMap(lambda x: (x[1])).distinct() # This list all the distinct genres from our dataset!
    
    getAllGenres_df = getAllGenres_rdd.toDF(["Title", "Genres array"])
    getAllGenres_df.printSchema()

    return fixedListOfGenres


# Add new columns, one for each genre, for each movie
def split_column_of_genres_string_in_array(dataset):
    ''' input: dataset as Dataframe
    output: genres_arr as one column (Dataframe)
    '''
    dataset_with_newcolumn = dataset.select(
        split(dataset.genres, '[|]').alias('genres_arr'))
    dataset_with_newcolumn = dataset.select(
        split(dataset.genres, '[|]').alias('genres_arr'))
    dataset_with_newcolumn.printSchema()
    return dataset_with_newcolumn


def split_col_genres_in_array_rdd(dataset):
    ''' Transform the genres column (string) into an array. Output as a rdd
    '''
    # index of genre in the dataset is 1
    dataset_as_rdd = dataset.rdd.map(lambda x: tuple(
        x[i] if i != 1 else x[1].split("|") for i in range(16)))
    return dataset_as_rdd


def processing_dataset_genres(dataset, initial_column_names):
    '''Takes dataset (where the genres column changed from type string -> string[] ) + list of genres (generated from dataset)
    and output the dataset (as dataframe) after applying StringIndexer
    '''
    test_all_genres = get_dataset_movie_genres(dataset) # Generate all the new columns to add to dataframe
    test_all_genres = test_all_genres.collect()
    print(test_all_genres)

    # dataset (as rdd) but the genres column String -> String[]
    test_rdd = split_col_genres_in_array_rdd(dataset)


    test_df = test_rdd.toDF(initial_column_names)  # turn dataset back to dataframe

    # adding the new columns
    test_df_2 = test_df
    for new_genre in test_all_genres:
        test_df_2 = test_df_2.withColumn(new_genre,
                        when(array_contains(test_df.genres, new_genre), lit(new_genre))
                        .otherwise(lit('n/a'))
                        )
    test_df_2.drop('genres')
    test_df_2.printSchema()
    # test_df_2.toPandas().to_csv('./genres_output/output-noah-result.csv')


    # StringIndexer
    indexed = test_df_2 
    for new_genre in test_all_genres:
        indexer = StringIndexer(inputCol=new_genre, outputCol='{g}_index'.format(g=new_genre))
        indexed = indexer.fit(indexed).transform(indexed)
        indexed = indexed.drop(new_genre)

    # indexed.show()

    return indexed, test_all_genres
    
# One Hot encoder
def one_hot_encoder_genres(test_all_genres, indexed):
    ''' Takes output of processing_dataset_genres() and apply one hot encoding
    See result in file "./genres_output/output-noah-index-encoded.csv"
    '''
    inputs = [genre + '_index' for genre in test_all_genres]
    outputs = [genre + '_ohe' for genre in test_all_genres]
    encoder = OneHotEncoder(inputCols=inputs, outputCols=outputs)
    model = encoder.fit(indexed)
    encoded = model.transform(indexed)
    return encoded, outputs

dataset = modify_dataset(dataset)
indexed, test_all_genres = processing_dataset_genres(dataset, initial_column_names)
dataset, encoded_column_names = one_hot_encoder_genres(test_all_genres,indexed)


root
 |-- Title: string (nullable = true)
 |-- Genres array: array (nullable = true)
 |    |-- element: string (containsNull = true)

['Action', 'Adventure', 'Fantasy', 'Sci-Fi', 'Thriller', 'Romance', 'Animation', 'Comedy', 'Family', 'Musical', 'Mystery', 'Western', 'Drama', 'History', 'Sport', 'Crime', 'Horror', 'War', 'Biography', 'Music', 'Documentary', 'Film-Noir']
root
 |-- title: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- production_companies: string (nullable = true)
 |-- vote_average: long (nullable = true)
 |-- revenue: string (nullable = true)
 |-- release_year: long (nullable = true)
 |-- release_month: long (nullable = true)
 |-- cast_number: long (nullable = true)
 |-- first_cast_gender: long (nullable = true)
 |-- second_cast_gender: long (nullable = true)
 |-- duration: long (nullable = true)
 |-- cast_total_facebook_likes: long (nullable = true)
 |-- content_rating: string (nullable = true)
 |-- bu

In [8]:
print(dataset.count()) # we end up with 3811 movies
print(dataset.printSchema())

3811
root
 |-- production_companies: string (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: integer (nullable = true)
 |-- revenue: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- release_month: integer (nullable = true)
 |-- cast_number: integer (nullable = true)
 |-- first_cast_gender: integer (nullable = true)
 |-- second_cast_gender: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- cast_total_facebook_likes: integer (nullable = true)
 |-- budget: long (nullable = true)
 |-- imdb_score: integer (nullable = true)
 |-- movie_facebook_likes: integer (nullable = true)
 |-- OHEContentIndex: vector (nullable = true)

None


In [9]:

# Converting profit to be represented on the scale: A-E

# Step 1:   Add a profit column. Remove revenue column.
#           Sort by profit (descending order).
dataset = (
    dataset.withColumn(
        "profit",
        (dataset["revenue"].cast(LongType()) - dataset["budget"].cast(LongType())),
    )
    .drop(*["revenue"])
    .sort("profit", ascending=False)
)
# Step 2:   Add an incremental ID in the "id" column to assign a grade.
#           Convert back to dataframe.
columns = dataset.columns
dataset = (
    dataset.rdd.zipWithIndex()
    .map(lambda x: (x[1],) + tuple(x[0]))
    .toDF(["id"] + columns)
)
# Step 3:   Assign a grade in the "profit_grade" column, corresponding to the value.
#           Remove id and profit columns.
count = dataset.count() // 5
dataset = (
    dataset.withColumn(
        "profit_grade",
        when((dataset.id >= 0) & (dataset.id < count), "A")
        .when((dataset.id >= (count + 1)) & (dataset.id < count * 2 + 1), "B")
        .when((dataset.id >= (count + 1) * 2) & (dataset.id < count * 3 + 2), "C")
        .when((dataset.id >= (count + 1) * 3) & (dataset.id < count * 4 + 3), "D")
        .otherwise("E"),
    ).drop(*["id", "profit"])
    # Randomize to "unsort" dataset.
    .orderBy(rand())
)

In [10]:
dataset = dataset.drop("production_companies","title","genres","duration","content_rating")

In [11]:

# Data prepration for the models

# Put all features in one vector
all_feature_cols = [item for item in dataset.columns if item != "profit_grade"]
assembler = VectorAssembler(inputCols=all_feature_cols, outputCol="userFeatures")
dataset = assembler.transform(dataset)


In [12]:
# Use RobusScaler to reduce outliers
scaler = RobustScaler(inputCol="userFeatures", outputCol="scaledFeatures",
                      withScaling=True, withCentering=False,
                      lower=0.25, upper=0.75)

scalerModel = scaler.fit(dataset)

# Transform each feature to have unit quantile range.
dataset = scalerModel.transform(dataset)

# Convert the label column to numeric format using StringIndexer
dataset = StringIndexer(inputCol="profit_grade", outputCol="indexedLabel").fit(dataset).transform(dataset)
#labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

In [13]:
normalizer = Normalizer(inputCol="scaledFeatures", outputCol="normalized_features", p=1.0)
dataset = normalizer.transform(dataset)

In [ ]:
#---Phase 3.a: Random Forest Model & Evaluation

In [375]:
# Random Forest Model

# Split data to training and test set
trainingData, testData = dataset.randomSplit([0.8, 0.2])

# Create a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol = "normalized_features", numTrees=16)

# Train model using the training data
model = rf.fit(trainingData)


# Evaluation

predictions = model.transform(testData)

predictions.select("prediction", "indexedLabel", "normalized_features").show(5)

# Find the accuracy of the model
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Random Forest Model accuracy = %g" % (accuracy))


+----------+------------+--------------------+
|prediction|indexedLabel| normalized_features|
+----------+------------+--------------------+
|       4.0|         4.0|(24,[1,2,3,4,5,6,...|
|       4.0|         4.0|(24,[1,2,3,4,5,6,...|
|       0.0|         0.0|(24,[0,1,2,3,4,5,...|
|       4.0|         4.0|(24,[0,1,2,3,4,5,...|
|       4.0|         4.0|(24,[0,1,2,3,4,5,...|
+----------+------------+--------------------+
only showing top 5 rows

0.4812760055478502


In [ ]:
#---Phase 3.b: Logistic Regression Model & Evaluation

In [ ]:
# Create the base Logistic Regression classifier.
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True,featuresCol='normalized_features', labelCol='indexedLabel')

# Create the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr,featuresCol='normalized_features', labelCol='indexedLabel')

grid = ParamGridBuilder().addGrid(lr.maxIter, [0, 1]).build()
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol='indexedLabel')
cv = CrossValidator(estimator=ovr, estimatorParamMaps=grid, evaluator=evaluator,parallelism=2)
cvModel = cv.fit(dataset)
acc = evaluator.evaluate(cvModel.transform(dataset))
print(acc)

In [376]:
# Logistic Regression Model using One Vs Rest

# Create the base Logistic Regression classifier.
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True,featuresCol='normalized_features', labelCol='indexedLabel')

# Create the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr,featuresCol='normalized_features', labelCol='indexedLabel')

# train the multiclass model.
ovrModel = ovr.fit(trainingData)

# Evaluation

predictions = ovrModel.transform(testData)

# Find the accuracy of the model
evaluator = MulticlassClassificationEvaluator(metricName="accuracy",predictionCol="prediction", labelCol='indexedLabel')
accuracy = evaluator.evaluate(predictions)
print("Logistic Regression Model accuracy = %g" % (accuracy))

Model accuracy = 0.406736
